In [1]:
import pandas as pd
from pypdf import PdfReader, PdfWriter
from ocrmypdf.exceptions import SubprocessOutputError
import os
import ocrmypdf
from tqdm import tqdm
from IPython.display import clear_output
import json
from unidecode import unidecode
import unicodedata
import re
import traceback
from collections import defaultdict
from lib.analyzer.document_analyzer import DocumentAnalyzer
from lib.analyzer.data_extractor import DataExtractor
import numpy as np
import spacy
from fuzzywuzzy import fuzz

/home/rafaeljosem/.pyenv/versions/3.11.7/envs/octane_project/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
skip_files = []
files_with_error = []
processed_files = []

raw_dir = './files/raw'
tmp_dir = './files/tmp'
output_dir = './files/ocr_address'
json_dir = './files/json/address'
sample_dir = './files/sample'
log_dir = './var'

In [3]:
for dirname in [raw_dir, tmp_dir, output_dir, json_dir, log_dir]:
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [ ]:
for name in os.listdir(raw_dir):
    src = os.path.join(raw_dir, name)
    reader = PdfReader(src)
        
    if len(reader.pages) <= 4:
        skip_files.append(name)
        continue
    
    page = reader.pages[reader.get_num_pages()-2]    
    dest = os.path.join(tmp_dir, name)
    
    #Extraer la penúltima página, donde se encuentra la dirección de la estación
    
    with open (os.path.join(tmp_dir, name), 'wb') as f:
        writer = PdfWriter(f)
        writer.add_page(page)
        writer.write(dest)

In [ ]:
import warnings

error_files = []
warning_files = []

for name in tqdm(os.listdir(tmp_dir), unit='file', desc='Files processed: '):
    
    #Omite los archivos que tienen cuatro páginas o menos
    if name in skip_files or os.path.exists(os.path.join(output_dir, name)):
        clear_output(wait=True)
        continue
    
    try:
        with warnings.catch_warnings(record = True) as w:
            
            warnings.simplefilter("always")
            #Realiza OCR
            ocrmypdf.ocr(os.path.join(tmp_dir, name),
                    os.path.join(output_dir, name),skip_text=True, language='spa', deskew=True)
            if w:
                raise w[-1].category(w[-1].message)
    
    except SubprocessOutputError as error:
        error_files.append({'file': name, 'error': error})
    
    except Warning as e:
        warning_files.append({'file': name, 'warning': e})
        
    clear_output(wait=True)

In [22]:
files_with_no_address = []

for name in os.listdir(output_dir):
    file = PdfReader(os.path.join(output_dir, name))
    text = file.pages[0].extract_text()
    
    if text.lower().find('reporte') < 0:
        files_with_no_address.append(name)
        os.remove(os.path.join(output_dir, name))
        continue
    
    
    

In [23]:
files_with_no_address

['2023 03 140.pdf',
 '2023 05 065.pdf',
 '2023 08 047.pdf',
 '2023 03 021.pdf',
 '2023 07 001.pdf',
 '2023 05 072.pdf',
 '2023 08 051.pdf',
 '2023 04 088.pdf',
 '2023 04 089.pdf',
 '2023 05 107.pdf',
 '2023 06 007.pdf',
 '2023 04 102.pdf',
 '2023 07 006.pdf',
 '2023 06 017.pdf',
 '2023 04 066.pdf',
 '2023 06 016.pdf',
 '2023 07 011.pdf',
 '2023 04 111.pdf',
 '2023 05 128.pdf',
 '2023 05 100.pdf',
 '2023 08 043.pdf',
 '2023 06 029.pdf',
 '2023 06 099.pdf',
 '2023 06 106.pdf',
 '2023 07 102.pdf',
 '2023 02 047.pdf',
 '2023 05 007.pdf',
 '2023 06 107.pdf',
 '2023 07 062.pdf',
 '2023 07 074.pdf',
 '2023 06 105.pdf',
 '2023 08 026.pdf',
 '2023 07 100.pdf',
 '2023 06 070.pdf',
 '2023 07 049.pdf',
 '2023 07 065.pdf',
 '2023 06 060.pdf',
 '2023 07 110.pdf',
 '2023 05 015.pdf',
 '2023 06 049.pdf',
 '2023 06 077.pdf',
 '2023 06 117.pdf',
 '2023 03 047.pdf',
 '2023 06 076.pdf',
 '2023 07 095.pdf',
 '2023 05 027.pdf',
 '2023 07 122.pdf',
 '2023 07 057.pdf',
 '2023 03 061.pdf',
 '2023 06 124.pdf',


In [24]:
len(files_with_no_address)

74

In [3]:
analyzer = DocumentAnalyzer()

results = analyzer.analyze(output_dir,json_dir )

Files processed: 100%|██████████| 736/736 [1:13:32<00:00,  6.00s/file]


In [7]:
results[:10]

[{'file_name': '2023 07 014.pdf',
  'result': {'api_version': '2023-07-31',
   'model_id': 'prebuilt-document',
   'content': 'aivepet\nAIVEPET, S.R.L-REP. DOMINICANA Reporte Fotográfico Ref. AIVEPET: 2023 07 014 Fecha de Inspección: 04/07/2023 E/S ATLANTIC PETROLEUM BONAO I AVENIDA ANIANA I ESQUINA PADRE BILLINI, BONAO, PROVINCIA MONSEÑOL NOUEL\nMEDICIÓN DEL CORTE DE AGUA LIBRE EN LOS TANQUES DE ALMACENAMIENTO\nMEDICIÓN DEL CORTE DE AGUA LIBRE EN LOS TANQUES DE ALMACENAMIENTO\nPrecios de los combustibles\nSEMANA DEL TAL 7\nWarlic --\nRD$291.60\nRD$273.50\n=\n=\nRD$237.10 =\nRD$220.60\n=\nCOMPARACIÓN DE PRECIOS PUBLICADOSPOR MICM VS LOS PUBLICADOS POR LA E/S\nPROCESO DE TOMA DE MUESTRAS EN LOS TANQUES DE ALMACENAMIENTO\n0012800\nNº DE SELLO DE LAS MUESTRAS RETENIDAS\nVISTA PANORÁMICA DE LA ESTACIÓN DE\nSERVICIO',
   'languages': [],
   'pages': [{'page_number': 1,
     'angle': 0.30776330828666687,
     'width': 8.5,
     'height': 11.0,
     'unit': 'inch',
     'lines': [{'content': 

In [5]:
addresses = []
for name in os.listdir(json_dir):    
    with open (os.path.join(json_dir, name), 'r') as f:
        address_data = json.load(f)
        extractor = DataExtractor(address_data)
        addresses.append({
            'file': name,
            'address': extractor.extract_address()
        })


In [120]:
addresses[:10]

[{'file': '2023 08 030.json',
  'address': 'TOTAL LA SUREÑA KM 2 1/2 CARRETERA SANCHEZ MUNICIPIO BANI- PROVINCIA PERAVÍA',
  'city': 'Peravia'},
 {'file': '2023 01 023.json',
  'address': 'ATLANTIC, BONAO AUTOPISTA DUARTE KM 77, BONAO. PROVINCIA MONSEÑOL NOUEL',
  'city': 'Monseñor Nouel'},
 {'file': '2023 07 008.json',
  'address': 'COMBUSDOM - CARRETERA YAMASÁ - PERALVILLO, EL CERCADILLO, YAMASÁ PROVINCIA MONTE PLATA.',
  'city': 'Monte Plata'},
 {'file': '2023 06 008.json',
  'address': 'NATIVA LA TORONJA- AVENIDA MARTÍN LUTHER KING, SANTO DOMINGO ESTE. PROVINCIA SANTO DOMINGO',
  'city': 'Santo Domingo Este'},
 {'file': '2023 05 144.json',
  'address': 'ECOPETRÓLEO EMMAR - CALLE SÁNCHEZ Nº 01. PROVINCIA ELIAS PIÑA',
  'city': 'Elías Piña'},
 {'file': '2023 05 001.json',
  'address': 'SHELL CAMBITA C/ DUARTE No.46, CAMBITA GARABITO - SAN CRISTÓBAL',
  'city': 'Duarte'},
 {'file': '2023 04 001.json',
  'address': 'TOTAL JIMENOA - AV. 16 DE AGOSTO Nº 47, JARABACOA- PROVINCIA LA VEGA',

In [61]:
spacy.prefer_gpu()

True

Funciones para extraer las ciudades haciendo uso de Spacy

In [84]:
# Load SpaCy Spanish large model
nlp = spacy.load("es_core_news_lg")
from spacy.matcher import Matcher


def remove_accents(input_str):
    return ''.join(
        c for c in unicodedata.normalize('NFD', input_str)
        if unicodedata.category(c) != 'Mn'
    )

def extract_city(address, cities):
    if not address:
        return None

    # Remove accents and lowercase
    address_normalized = remove_accents(address.lower())

    # Use SpaCy to process the address
    doc = nlp(address_normalized)

    # Extract potential cities using NER
    potential_cities = [ent.text for ent in doc.ents if ent.label_ in ("LOC", "GPE")]

    # Add provincia and municipio matches
    provincia_match = re.search(r'provincia\s*-*\s*([a-zá-úñ\s-]+)', address_normalized)
    if provincia_match:
        potential_cities.append(provincia_match.group(1).strip())

    municipio_match = re.search(r'municipio\s*-*\s*([a-zá-úñ\s-]+)', address_normalized)
    if municipio_match:
        potential_cities.append(municipio_match.group(1).strip())

    # Add rule-based matching
    matcher = Matcher(nlp.vocab)
    for city in cities:
        city_pattern = [{"LOWER": remove_accents(city.lower())}]
        matcher.add(city, [city_pattern])

    matches = matcher(doc)
    for match_id, start, end in matches:
        potential_cities.append(doc[start:end].text)

    # Check for "Santo Domingo Este/Norte/Oeste" specifically
    for specific_sd in ["santo domingo este", "santo domingo norte", "santo domingo oeste"]:
        if specific_sd in address_normalized:
            return specific_sd.title()

    # Use FuzzyWuzzy for similarity matching
    best_match = None
    best_score = 0
    for city in cities:
        for potential_city in potential_cities:
            score = fuzz.partial_ratio(remove_accents(city.lower()), remove_accents(potential_city.lower()))
            if score > best_score:
                # Avoid matching 'Duarte' when it's part of 'Autopista Duarte', 'Calle Duarte', 'Carretera Duarte' or 'Avenida Duarte'
                if city.lower() == "duarte" and any(keyword in address_normalized for keyword in ["autopista duarte", "calle duarte", "carretera duarte", "avenida duarte"]):
                    continue
                # Avoid matching 'Independencia' as city if it's part of an avenue name
                if city.lower() == "independencia" and "avenida independencia" in address_normalized:
                    continue
                best_match = city
                best_score = score

    # Consider a threshold to ensure a good match
    if best_score >= 75:  # Adjust threshold as needed
        return best_match.title()

    return None


In [85]:
# The list of cities
cities = [
    "Azua", "Baoruco", "Barahona", "Dajabón", "Distrito Nacional", "Duarte", 
    "Elías Piña", "El Seibo", "Espaillat", "Hato Mayor", "Hermanas Mirabal", 
    "Independencia", "La Altagracia", "La Romana", "La Vega", 
    "María Trinidad Sánchez", "Monseñor Nouel", "Monte Cristi", "Monte Plata", 
    "Pedernales", "Peravia", "Puerto Plata", "Samaná", "San Cristóbal", 
    "San José de Ocoa", "San Juan", "San Pedro de Macorís", "Sánchez Ramírez", 
    "Santiago", "Santiago Rodríguez", "Santo Domingo", "Valverde",
    "Santo Domingo Este", "Santo Domingo Norte", "Santo Domingo Oeste"
]



# Extract cities
for item in tqdm(addresses):
    city = extract_city(item.get('address'), cities)
    item['city'] = city

100%|██████████| 736/736 [00:04<00:00, 151.28it/s]


In [86]:
df = pd.DataFrame(addresses)

In [80]:
df[:10]

,file,address,city
0,2023 08 030.json,TOTAL LA SUREÑA KM 2 1/2 CARRETERA SANCHEZ MUN...,Peravia
1,2023 01 023.json,"ATLANTIC, BONAO AUTOPISTA DUARTE KM 77, BONAO....",Monseñor Nouel
2,2023 07 008.json,"COMBUSDOM - CARRETERA YAMASÁ - PERALVILLO, EL ...",Monte Plata
3,2023 06 008.json,"NATIVA LA TORONJA- AVENIDA MARTÍN LUTHER KING,...",Santo Domingo Este
4,2023 05 144.json,ECOPETRÓLEO EMMAR - CALLE SÁNCHEZ Nº 01. PROVI...,Elías Piña
5,2023 05 001.json,"SHELL CAMBITA C/ DUARTE No.46, CAMBITA GARABIT...",Duarte
6,2023 04 001.json,"TOTAL JIMENOA - AV. 16 DE AGOSTO Nº 47, JARABA...",La Vega
7,2023 01 074.json,TDC-TRANS DIESEL DEL CARIBE AV. MONUMENTAL Nº ...,Duarte
8,2023 02 092.json,"SHELL OVANDO - AV NICOLAS DE OVANDO ESQ, 41 CR...",None
9,2023 04 056.json,SUNIX RÍO SAN JUAN -AUTOPISTA CABRERA -RÍO SAN...,María Trinidad Sánchez


Guardamos en excel para hacer una revisión manual

In [88]:
df.to_excel('./addresses.xlsx', index=False)    

Cargamos los datos corregidos en excel, así como el dataframe original, para unirlos.

In [45]:
df_address = pd.read_excel('./addresses.xlsx')
# df = pd.read_parquet('./gas_station_dataset.parquet').drop(columns=['CIUDAD', 'Latitud', 'Longitud'])


In [46]:
df.columns

Index(['ANÁLISIS', 'UNIDADES', 'MIN', 'MAX', 'MÉTODO', 'RESULTADOS',
       'NOMBRE ESTACIÓN', 'PRODUCTO', 'FECHA', 'NUM PAGINA', 'ARCHIVO',
       'INCERTIDUMBRE/ UNCERTAINTY %', 'DIRECCIÓN', 'MARCA ESTACION',
       'OBSERVACIONES', 'CONFORMIDAD'],
      dtype='object')

Los datos serán unidos por medio de la columna archivo. Como la columna 'file' del dataframe que contiene las direcciones tiene la extensión json, procedemos a cambiarlo a pdf.

In [47]:
df_address['file'] = df_address['file'].apply(lambda x: x.replace('.json', '.pdf'))

Hacemos el merge

In [48]:
df = df.merge(right=df_address, left_on='ARCHIVO', right_on='file')

Limpiamos el dataframe

In [49]:
df['DIRECCIÓN'] = df['address']
df.drop(columns=['file', 'address'], inplace=True)
df.rename(columns={'city': 'CIUDAD'}, inplace=True)

In [50]:
df.head()

,ANÁLISIS,UNIDADES,MIN,MAX,MÉTODO,RESULTADOS,NOMBRE ESTACIÓN,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %,DIRECCIÓN,MARCA ESTACION,OBSERVACIONES,CONFORMIDAD,CIUDAD,Latitud,Longitud
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,ATLANTIC PETROLEUM BONAO I AVENIDA ANIANA I ES...,ATLANTIC,Observación: Valores resaltados se encuentran ...,False,Monseñor Nouel,18.92721,-70.39728
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,ATLANTIC PETROLEUM BONAO I AVENIDA ANIANA I ES...,ATLANTIC,Observación: Valores resaltados se encuentran ...,False,Monseñor Nouel,18.92721,-70.39728
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,ATLANTIC PETROLEUM BONAO I AVENIDA ANIANA I ES...,ATLANTIC,Observación: Valores resaltados se encuentran ...,False,Monseñor Nouel,18.92721,-70.39728
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,ATLANTIC PETROLEUM BONAO I AVENIDA ANIANA I ES...,ATLANTIC,Observación: Valores resaltados se encuentran ...,False,Monseñor Nouel,18.92721,-70.39728
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,ATLANTIC PETROLEUM BONAO I AVENIDA ANIANA I ES...,ATLANTIC,Observación: Valores resaltados se encuentran ...,False,Monseñor Nouel,18.92721,-70.39728


In [51]:
df.to_parquet('./gas_station_dataset.parquet', index=False)
df.to_csv('./gas_station_dataset.csv', index=False)
df.to_json('./gas_station_dataset.json', orient='records', index=False)